In [1]:
import numpy as np
import librosa
import IPython.display as ipd
import random
import scipy.stats as st
import scipy.special as ss

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
y, sr = librosa.load('audio/NickDrake_8k.wav')
z, _= librosa.load('audio/Loureed_8k.wav')
w, _= librosa.load('audio/tom1_8k.wav')
T = len(y)
S = np.array([z,w,y])
theta = np.array([np.pi/2, np.pi-np.pi/8, np.pi/8])/2
A = np.array([np.sin(theta), np.cos(theta),[0.19509032, 0.70710678, 0.98078528]])
m , n = A.shape
sigma = 0.01
N = st.norm.rvs(loc = 0, scale = sigma, size = (m,T) )
X  = np.dot(A,S) + N

In [5]:
A

array([[0.70710678, 0.98078528, 0.19509032],
       [0.70710678, 0.19509032, 0.98078528],
       [0.19509032, 0.70710678, 0.98078528]])

In [6]:
from scipy.fftpack import dct, idct

CX = dct(X, norm = 'ortho')
CX.shape

(3, 180634)

In [7]:
np.linalg.det(A)

-0.7575026314470591

In [8]:
A.shape

(3, 3)

In [9]:
#%%% Priors %%%

#% Inverse-Gamma prior on sigma_sqrd
shape_sigma_sqrd_prior = 1
scale_sigma_sqrd_prior = 1

#% Gamma prior on lambda    
shape_lambda_prior = np.zeros((n,1))
scale_lambda_prior = np.zeros((n,1))

#% Exponential prior on alpha
scale_alpha_prior = np.ones((n,1))

shape_v_prior = np.ones((n,1))
scale_v_prior = np.ones((T,1))     #vamos gerar uma matriz n*T, onde cada linha representa V_i,k 


#%% Gaussian prior on the columns of A
#%sigma_A = 100 * ones(n,1)

#%%% Initializations %%%
cur_A = np.array([[1, 1, 1],[0, 0, 0]])
#cur_A = np.identity(n)

#% Variance of the noise initialisation
#%cur_sigma_sqrd=(sigma)^2
cur_sigma_sqrd=0.1

#% Sources scale parameter initialisation
cur_lambda=1*np.ones((n,1))
#%cur_beta = betaw

cur_alpha = 0.5*np.ones((n,1))

cur_v = 0.5*np.ones((n,T))

#% Sources coefficients initialisation  S
cur_Source_C= 0.5*np.ones((n,T))


SkXsk = np.ones((n,n))
Sigma_sk = np.ones((T,n,n))
aux = np.ones((T,n,n))
Mu_sk = np.ones((T,n))
              
              
mu_A = np.ones((m,n))
cur_A = np.ones((m,n))
sigma_A =np.ones((n,n))
aux2_v = np.ones((n,T))

In [10]:
cur_A

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [11]:
def p(x,i_n):
    return ((np.log(P_i[i_n])*(-x/2 - 1) + (x*T/2)*np.log(x*cur_lambda[i_n]/2) - np.log(ss.gamma(x/2)))*T + (-x*cur_lambda[i_n]*R_i[i_n]/2))
    
def q(x,x2):
    return st.gamma.logpdf(x,a=1,loc = x2)
def MH(x,x_star,p,q,i_n):
    return min([1,np.exp(p(x_star,i_n) + q(x,x_star) - p(x,i_n) - q(x_star,x))])

In [12]:
import time

In [ ]:
# separa os list comprehent alocando memoria antecipadamente

usar numba e jit para tentar otimizar os códigos. Passar as variáveis em função para amostrar  

fazer o teste com três observações, problema com solução única

In [14]:
alpha_sigma = (T-n)*m/2
SkXsk[::] = np.linalg.pinv(np.sum([np.outer(cur_Source_C[:,k],cur_Source_C[:,k].T) for k in range(T)],axis = 0))   #inverse matrix sum sk*skT 
X_times_s = np.array([[CX[k,j]*cur_Source_C[:,j] for j in range(T)] for k in range(m)])
print(X_times_s.shape)


(3, 180634, 3)


In [15]:
X_times_s2 = np.array([np.sum(k,axis = 0 )for k in X_times_s ]) #soma de cada x_k_i*sk
print(X_times_s2.shape)


(3, 3)


In [18]:
SkXsk[::] = np.linalg.pinv(np.sum([np.outer(cur_Source_C[:,k],cur_Source_C[:,k].T) for k in range(T)],axis = 0))   #inverse matrix sum sk*skT 
X_times_s = np.array([[CX[k,j]*cur_Source_C[:,j] for j in range(T)] for k in range(m)])
X_times_s = np.array([np.sum(k,axis = 0 )for k in X_times_s ])

In [21]:
 np.array([(np.matrix(SkXsk)*k.reshape(-1,1)).reshape(1,-1) for k in X_times_s ]).reshape(m,n)

array([[ 1.08257501e-05,  1.08257501e-05,  1.08257501e-05],
       [-4.65618412e-05, -4.65618412e-05, -4.65618412e-05],
       [ 1.03575733e-05,  1.03575733e-05,  1.03575733e-05]])

In [6]:
start_time = time.time()
n_iter = 1
iter_MH = 1
for i in range(n_iter):
        print(i)
  
        #amostrando s^till
        aux =  (1/cur_sigma_sqrd)*np.dot(A.T,A)

        Sigma_sk[::] = np.array([np.linalg.pinv(aux + np.diag(1/cur_v[:,k]))    for k in range(T)])    # um sobre diag
        aux = [np.dot(A.T,CX[:,k]) for k in range(T)]
        Mu_sk[::] = np.array([(1/cur_sigma_sqrd)*np.dot(Sigma_sk[k],aux[k]) for k in range(T)])
        cur_Source_C[:,:] = np.array([st.multivariate_normal.rvs(Mu_sk[k],Sigma_sk[k]) for k in range(T)]).T
        #amostrando sigma^2

        alpha_sigma = (T-n)*m/2
        SkXsk[::] = np.linalg.inv(np.sum([np.outer(cur_Source_C[:,k],cur_Source_C[:,k].T) for k in range(T)],axis = 0))   #inverse matrix sum sk*skT 
        X_times_s = np.array([[CX[k,j]*cur_Source_C[:,j] for j in range(T)] for k in range(m)])
        X_times_s = np.array([np.sum(k,axis = 0 )for k in X_times_s ]) #soma de cada x_k_i*sk

        beta_sigma = 2/(np.sum(np.power(CX,2)) - sum([float(k*np.matrix(SkXsk)*k.reshape(-1,1)) for k in X_times_s ]))

        cur_sigma_sqrd= st.invgamma.rvs(a = alpha_sigma, scale = beta_sigma)

        #amostrando A
        sigma_A[::] = cur_sigma_sqrd*SkXsk
        mu_A[::] = np.array([(np.matrix(SkXsk)*k.reshape(-1,1)).reshape(1,-1) for k in X_times_s ]).reshape(m,n)
        cur_A[::] = np.array([st.multivariate_normal.rvs(mean = k, cov = sigma_A ) for k in mu_A])


        #amostrando V
        shape_v_prior = (cur_alpha + 1)/2
        aux_v = np.multiply(cur_alpha,cur_lambda)
        aux2_v[::] = np.power(cur_Source_C,2)
        cur_v[::] = np.array([[st.invgamma.rvs(a = shape_v_prior[ii], scale = 2/(aux_v[ii] + aux2_v[ii][k])  ) for k in range(T)] for ii in range(n)])

        #amostrando lambda
        R_i = (np.sum(1/cur_v, axis= 1))
        P_i = (np.prod(cur_v, axis= 1))
        shape_lambda_prior[::] = cur_alpha*T/2 
        scale_lambda_prior[::] = 2/np.multiply(R_i,cur_alpha.T).T
        cur_lambda[::] = st.gamma.rvs(a = shape_lambda_prior, scale = scale_lambda_prior)



        #amostrando alpha
        w = np.zeros(iter_MH+1)
        for j in range(n):
            for ii in range(iter_MH):
                u = st.uniform.rvs()
                x_star = st.gamma.rvs(a = 1,loc = w[ii])
            if u < MH(w[ii],x_star,p,q,j):
                w[ii+1] = x_star
            else:
                w[ii+1] = w[ii]

            cur_alpha[j] = w[-1]
            
    
print("--- %s seconds ---" % (time.time() - start_time))
   

0
--- 41.551392555236816 seconds ---
--- 41.551392555236816 seconds ---


In [9]:
SkXsk.shape

(3, 3)

In [17]:
cur_A - A

array([[-2.11033886e-04, -1.32971148e-04, -2.42603020e-04],
       [-6.88896447e-05, -1.66531197e-04, -1.16210037e-05]])

In [20]:
idct(cur_Source_C)

array([[-57.75006347, -32.54049617,  67.10129398, ...,  12.10911812,
         19.44243983, -39.4957884 ],
       [ 18.37593941, -29.57945293, -20.12494498, ..., -95.44336038,
        -37.53418129,  60.19361519],
       [ 10.44625233,  23.0587494 ,  -8.72004101, ...,   2.9676937 ,
          9.03522244, -21.62732573]])

In [11]:
np.linalg.inv(np.diag(cur_v[:,0]))

array([[18.5149899 ,  0.        ,  0.        ],
       [ 0.        ,  7.91501017,  0.        ],
       [ 0.        ,  0.        ,  0.35032513]])

In [12]:
np.diag(1/cur_v[:,0])

array([[18.5149899 ,  0.        ,  0.        ],
       [ 0.        ,  7.91501017,  0.        ],
       [ 0.        ,  0.        ,  0.35032513]])

In [18]:
ipd.Audio(idct(cur_Source_C)[0], rate=sr)


In [19]:
ipd.Audio(idct(cur_Source_C)[2], rate=sr)


In [20]:
ipd.Audio(idct(cur_Source_C)[1], rate=sr)


In [55]:
x_star = st.gamma.rvs(a = 1,loc = w[ii])
x_star

cur_lambda
np.log(P_i)

<ipython-input-55-84dbe4b10b70>:5: RuntimeWarning: divide by zero encountered in log
  np.log(P_i)


array([-inf,  inf, -inf])

In [49]:
        w = np.ones(iter_MH+1)
        for j in range(n):
            for ii in range(iter_MH):
                u = st.uniform.rvs()
                x_star = st.gamma.rvs(a = 1,loc = w[ii])
            if u < MH(w[ii],x_star,p,q,j):
                w[ii+1] = x_star
            else:
                w[ii+1] = w[ii]

            cur_alpha[j] = w[-1]

<ipython-input-40-f7051a195c25>:2: RuntimeWarning: divide by zero encountered in log
  return ((np.log(P_i[i_n])*(-x/2 - 1) + (x*T/2)*np.log(x*cur_lambda[i_n]/2) - np.log(ss.gamma(x/2)))*T + (-x*cur_lambda[i_n]*R_i[i_n]/2))


TypeError: _parse_args() missing 1 required positional argument: 'a'

In [58]:
shape_lambda_prior[::] = cur_alpha*T/2 
scale_lambda_prior[::] = 2/np.multiply((np.sum(1/cur_v, axis= 1)),cur_alpha.T).T
cur_lambda[::] = st.gamma.rvs(a = shape_lambda_prior, scale = scale_lambda_prior)

In [53]:
2/np.multiply((np.sum(1/cur_v, axis= 1)),cur_alpha.T)

array([[7.87219289e-05, 7.88460829e-05, 7.88376617e-05]])

In [55]:
shape_lambda_prior[::] = cur_alpha*T/2 
scale_lambda_prior[::] = 2/np.multiply((np.sum(1/cur_v, axis= 1)),cur_alpha.T).T

In [34]:
shape_lambda_prior

array([[45158.5],
       [45158.5],
       [45158.5]])

In [18]:
shape_v_prior = (cur_alpha + 1)/2
aux_v = np.multiply(cur_alpha,cur_lambda)
aux2_v = np.power(cur_Source_C,2)
V_test = np.array([[st.invgamma.rvs(a = shape_v_prior[ii], scale = 2/(aux_v[ii] + aux2_v[ii][k])  ) for k in range(T)] for ii in range(n)])

array([ 3.48922661,  4.81414987,  1.19238353, ..., 52.13948101,
       56.43760375,  1.63100255])

In [152]:
cur_A = np.array([[1, 1, 1],[0, 0, 0]])

In [174]:
cur_A[:,:] = np.array([[1, 1, 1],[0, 0, 0]])

In [177]:
cur_A = np.array([st.multivariate_normal.rvs(mean = k, cov = sigma_A ) for k in mu_A])


In [178]:
cur_A

array([[0.30251286, 0.32252155, 0.17742992],
       [0.28323298, 0.17297806, 0.29774672]])

In [79]:
beta_sigma

0.00018953831564930468

In [53]:
#np.sum([np.outer(cur_Source_C[:,k],cur_Source_C[:,k].T) for k in range(T)],axis = 0)   # sum sk*skT 
X_times_s = np.array([[CX[k,j]*cur_Source_C[:,j] for j in range(T)] for k in range(m)])
X_times_s = np.array([np.sum(k,axis = 0 )for k in X_times_s ]) #soma de cada x_k_i*sk


array([[4.35334601e-05, 2.65624157e-05, 2.73029274e-05],
       [2.65624157e-05, 4.69567728e-05, 8.17756045e-06],
       [2.73029274e-05, 8.17756045e-06, 4.81241542e-05]])

In [81]:
[(np.matrix(SkXsk)*k.reshape(-1,1)).T for k in X_times_s ]

[matrix([[0.30013997, 0.32049972, 0.17787411]]),
 matrix([[0.28312749, 0.17349248, 0.29676119]])]

In [75]:
sum([float(k*np.matrix(SkXsk)*k.reshape(-1,1)) for k in X_times_s ])

5083.492056040932

In [55]:
CX.shape
X_times_s[0]

array([3437.80123426, 4714.4278433 ,  957.84096596])

In [49]:
[np.sum(k,axis = 0 )for k in X_times_s ]

[9076.301651636952, 8484.260360777864]

In [79]:
np.sum(np.power(CX,2)) - sum([float(k*np.matrix(SkXsk)*k.reshape(-1,1)) for k in X_times_s ])


15577.198244530475

In [195]:
np.linalg.inv(np.matrix([[1,-3/4,0],[-1/2,1,-1/2],[0,-3/4,1]]))

matrix([[2.5, 3. , 1.5],
        [2. , 4. , 2. ],
        [1.5, 3. , 2.5]])

In [193]:
np.matrix([[1,-3/4,1],[-1/2,1,-1/2],[0,-3/4,1]])

matrix([[ 1.  , -0.75,  1.  ],
        [-0.5 ,  1.  , -0.5 ],
        [ 0.  , -0.75,  1.  ]])